# Task 2: Quantitative Analysis using PyNance and TA-Lib

**Financial News Sentiment Analysis - Week 1 Challenge**

This notebook performs quantitative analysis on stock price data using:
- **yfinance**: Download stock price data
- **TA-Lib**: Calculate technical indicators (MA, RSI, MACD)
- **PyNance**: Financial metrics and analysis
- **Visualizations**: Understand data and impact of indicators


## 1. Setup and Imports


In [ ]:
# Standard library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime, timedelta
import yfinance as yf

# Technical Analysis
import talib

# PyNance for financial metrics
try:
    import pynance as pn
    PYNNANCE_AVAILABLE = True
except ImportError:
    PYNNANCE_AVAILABLE = False
    print("⚠️  PyNance not available. Some features will be limited.")
    print("   Install with: pip install pynance")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
warnings.filterwarnings('ignore')

# Set paths
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT / 'data'
FIGURES_DIR = PROJECT_ROOT / 'figures'

# Create directories if they don't exist
FIGURES_DIR.mkdir(exist_ok=True)

print("✅ Setup complete!")
print(f"TA-Lib version: {talib.__version__ if hasattr(talib, '__version__') else 'installed'}")
if PYNNANCE_AVAILABLE:
    print(f"PyNance available: {PYNNANCE_AVAILABLE}")


## 2. Load Stock Price Data

We'll use yfinance to download stock price data. You can either:
1. Use stocks from the news dataset
2. Specify stocks manually


In [ ]:
# Option 1: Load stocks from news dataset (if available)
news_data_files = list(DATA_DIR.glob('*.csv')) + list(DATA_DIR.glob('*.json'))

stocks_to_analyze = []
if news_data_files:
    try:
        news_file = news_data_files[0]
        if news_file.suffix == '.csv':
            news_df = pd.read_csv(news_file, low_memory=False)
        else:
            news_df = pd.read_json(news_file)
        
        if 'stock' in news_df.columns:
            # Get top stocks by article count
            top_stocks = news_df['stock'].value_counts().head(10).index.tolist()
            stocks_to_analyze = top_stocks[:5]  # Analyze top 5 stocks
            print(f"📊 Found stocks in news data: {len(news_df['stock'].unique())} unique stocks")
            print(f"Selected top 5 stocks: {stocks_to_analyze}")
    except Exception as e:
        print(f"⚠️  Could not load news data: {e}")

# Option 2: Manual stock selection (if no news data or to override)
if not stocks_to_analyze:
    # Default stocks for demonstration
    stocks_to_analyze = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
    print(f"📊 Using default stocks: {stocks_to_analyze}")

print(f"\n🎯 Stocks to analyze: {stocks_to_analyze}")


In [ ]:
# Download stock price data using yfinance
# Set date range (default: last 2 years)
end_date = datetime.now()
start_date = end_date - timedelta(days=730)  # 2 years

print(f"📥 Downloading stock data from {start_date.date()} to {end_date.date()}...")
print("This may take a few moments...\n")

stock_data = {}

for ticker in stocks_to_analyze:
    try:
        print(f"Downloading {ticker}...", end=" ")
        ticker_obj = yf.Ticker(ticker)
        df = ticker_obj.history(start=start_date, end=end_date)
        
        if not df.empty:
            # Standardize column names (yfinance uses capital letters)
            df.columns = [col.lower() for col in df.columns]
            df.index.name = 'date'
            df = df.reset_index()
            
            # Ensure we have required columns
            required_cols = ['date', 'open', 'high', 'low', 'close', 'volume']
            if all(col in df.columns for col in required_cols):
                stock_data[ticker] = df
                print(f"✅ {len(df)} records")
            else:
                print(f"❌ Missing required columns")
        else:
            print(f"❌ No data available")
    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✅ Successfully downloaded data for {len(stock_data)} stocks")


In [ ]:
# Display sample data
if stock_data:
    sample_ticker = list(stock_data.keys())[0]
    sample_df = stock_data[sample_ticker]
    
    print(f"📊 Sample data for {sample_ticker}:")
    print(f"Shape: {sample_df.shape[0]:,} rows × {sample_df.shape[1]} columns")
    print(f"\nDate range: {sample_df['date'].min().date()} to {sample_df['date'].max().date()}")
    print(f"\nFirst few rows:")
    display(sample_df.head())
    print(f"\nData types:")
    display(sample_df.dtypes)
    print(f"\nBasic statistics:")
    display(sample_df[['open', 'high', 'low', 'close', 'volume']].describe())


## 3. Prepare Data for Technical Analysis

Ensure data is properly formatted for TA-Lib calculations.


In [ ]:
def prepare_data_for_talib(df):
    """
    Prepare DataFrame for TA-Lib calculations.
    TA-Lib requires numpy arrays with specific data types.
    """
    df = df.copy()
    
    # Ensure date is datetime
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
    
    # Sort by date
    df = df.sort_index()
    
    # Ensure required columns exist and are numeric
    required_cols = ['open', 'high', 'low', 'close', 'volume']
    for col in required_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Remove any rows with NaN in critical columns
    df = df.dropna(subset=['open', 'high', 'low', 'close'])
    
    return df

# Prepare all stock data
prepared_data = {}
for ticker, df in stock_data.items():
    prepared_data[ticker] = prepare_data_for_talib(df)
    print(f"✅ Prepared {ticker}: {len(prepared_data[ticker])} records")

print(f"\n📊 All data prepared for technical analysis!")


## 4. Calculate Technical Indicators with TA-Lib

### 4.1 Moving Averages (MA)


In [ ]:
def calculate_technical_indicators(df):
    """
    Calculate various technical indicators using TA-Lib.
    """
    df = df.copy()
    
    # Convert to numpy arrays for TA-Lib
    high = df['high'].values.astype(float)
    low = df['low'].values.astype(float)
    close = df['close'].values.astype(float)
    open_price = df['open'].values.astype(float)
    volume = df['volume'].values.astype(float)
    
    # Moving Averages
    df['SMA_20'] = talib.SMA(close, timeperiod=20)
    df['SMA_50'] = talib.SMA(close, timeperiod=50)
    df['SMA_200'] = talib.SMA(close, timeperiod=200)
    df['EMA_12'] = talib.EMA(close, timeperiod=12)
    df['EMA_26'] = talib.EMA(close, timeperiod=26)
    
    # RSI (Relative Strength Index)
    df['RSI'] = talib.RSI(close, timeperiod=14)
    
    # MACD (Moving Average Convergence Divergence)
    macd, macd_signal, macd_hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_signal'] = macd_signal
    df['MACD_hist'] = macd_hist
    
    # Bollinger Bands
    bb_upper, bb_middle, bb_lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['BB_upper'] = bb_upper
    df['BB_middle'] = bb_middle
    df['BB_lower'] = bb_lower
    
    # Stochastic Oscillator
    slowk, slowd = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
    df['Stoch_K'] = slowk
    df['Stoch_D'] = slowd
    
    # Average True Range (ATR)
    df['ATR'] = talib.ATR(high, low, close, timeperiod=14)
    
    # On Balance Volume (OBV)
    df['OBV'] = talib.OBV(close, volume)
    
    # Average Directional Index (ADX)
    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    
    return df

# Calculate indicators for all stocks
indicators_data = {}
for ticker, df in prepared_data.items():
    print(f"Calculating indicators for {ticker}...", end=" ")
    indicators_data[ticker] = calculate_technical_indicators(df)
    print(f"✅")

print(f"\n✅ Technical indicators calculated for {len(indicators_data)} stocks")


In [ ]:
# Display sample indicators
if indicators_data:
    sample_ticker = list(indicators_data.keys())[0]
    sample_df = indicators_data[sample_ticker]
    
    print(f"📊 Technical Indicators for {sample_ticker}:")
    print(f"\nAvailable indicators:")
    indicator_cols = [col for col in sample_df.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
    print(f"  {', '.join(indicator_cols)}")
    
    print(f"\nSample data (last 5 rows):")
    display(sample_df[['close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'MACD_signal']].tail())


## 5. Financial Metrics with PyNance

PyNance provides additional financial analysis capabilities.


In [ ]:
def calculate_pynance_metrics(df, ticker):
    """
    Calculate financial metrics using PyNance.
    Note: PyNance may have different API, adjust based on actual library.
    """
    metrics = {}
    
    if not PYNNANCE_AVAILABLE:
        # Fallback: Calculate basic metrics manually
        df = df.copy()
        
        # Daily returns
        df['daily_return'] = df['close'].pct_change()
        
        # Cumulative returns
        df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1
        
        # Volatility (rolling 30-day)
        df['volatility_30d'] = df['daily_return'].rolling(window=30).std() * np.sqrt(252)
        
        # Sharpe ratio (annualized, assuming risk-free rate of 0.02)
        risk_free_rate = 0.02
        excess_returns = df['daily_return'] - (risk_free_rate / 252)
        df['sharpe_ratio'] = (excess_returns.rolling(window=252).mean() / 
                             excess_returns.rolling(window=252).std()) * np.sqrt(252)
        
        # Maximum drawdown
        cumulative = (1 + df['daily_return']).cumprod()
        running_max = cumulative.expanding().max()
        df['drawdown'] = (cumulative - running_max) / running_max
        df['max_drawdown'] = df['drawdown'].expanding().min()
        
        metrics = {
            'total_return': df['cumulative_return'].iloc[-1] if len(df) > 0 else 0,
            'volatility': df['volatility_30d'].iloc[-1] if len(df) > 0 else 0,
            'sharpe_ratio': df['sharpe_ratio'].iloc[-1] if len(df) > 0 else 0,
            'max_drawdown': df['max_drawdown'].iloc[-1] if len(df) > 0 else 0,
            'avg_daily_return': df['daily_return'].mean(),
            'data': df
        }
    else:
        # Use PyNance if available
        try:
            # PyNance usage (adjust based on actual API)
            # This is a placeholder - adjust based on actual PyNance documentation
            metrics = {
                'note': 'PyNance integration - adjust based on actual API',
                'data': df
            }
        except Exception as e:
            print(f"⚠️  PyNance error: {e}")
            metrics = {'data': df}
    
    return metrics

# Calculate PyNance metrics for all stocks
pynance_metrics = {}
for ticker, df in indicators_data.items():
    print(f"Calculating PyNance metrics for {ticker}...", end=" ")
    pynance_metrics[ticker] = calculate_pynance_metrics(df, ticker)
    print(f"✅")

print(f"\n✅ Financial metrics calculated!")


In [ ]:
# Display financial metrics summary
if pynance_metrics:
    metrics_summary = []
    for ticker, metrics in pynance_metrics.items():
        if 'total_return' in metrics:
            metrics_summary.append({
                'Ticker': ticker,
                'Total Return (%)': f"{metrics['total_return']*100:.2f}%",
                'Volatility (%)': f"{metrics['volatility']*100:.2f}%" if metrics['volatility'] else 'N/A',
                'Sharpe Ratio': f"{metrics['sharpe_ratio']:.2f}" if metrics['sharpe_ratio'] else 'N/A',
                'Max Drawdown (%)': f"{metrics['max_drawdown']*100:.2f}%" if metrics['max_drawdown'] else 'N/A',
                'Avg Daily Return (%)': f"{metrics['avg_daily_return']*100:.3f}%"
            })
    
    if metrics_summary:
        metrics_df = pd.DataFrame(metrics_summary)
        print("📊 Financial Metrics Summary:")
        display(metrics_df)


## 6. Visualizations

Create comprehensive visualizations to understand the data and impact of indicators.


In [ ]:
def plot_stock_with_indicators(df, ticker, save_path=None):
    """
    Create comprehensive visualization of stock price with technical indicators.
    """
    fig = plt.figure(figsize=(16, 12))
    gs = fig.add_gridspec(4, 1, height_ratios=[3, 1, 1, 1], hspace=0.3)
    
    # 1. Price with Moving Averages
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(df.index, df['close'], label='Close Price', linewidth=2, color='black')
    ax1.plot(df.index, df['SMA_20'], label='SMA 20', linewidth=1.5, alpha=0.7)
    ax1.plot(df.index, df['SMA_50'], label='SMA 50', linewidth=1.5, alpha=0.7)
    ax1.plot(df.index, df['SMA_200'], label='SMA 200', linewidth=1.5, alpha=0.7)
    ax1.fill_between(df.index, df['BB_upper'], df['BB_lower'], alpha=0.2, label='Bollinger Bands')
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1.set_title(f'{ticker} - Stock Price with Moving Averages and Bollinger Bands', fontsize=14, fontweight='bold')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # 2. RSI
    ax2 = fig.add_subplot(gs[1])
    ax2.plot(df.index, df['RSI'], label='RSI', linewidth=1.5, color='purple')
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
    ax2.fill_between(df.index, 30, 70, alpha=0.1, color='gray')
    ax2.set_ylabel('RSI', fontsize=12)
    ax2.set_ylim(0, 100)
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    # 3. MACD
    ax3 = fig.add_subplot(gs[2])
    ax3.plot(df.index, df['MACD'], label='MACD', linewidth=1.5, color='blue')
    ax3.plot(df.index, df['MACD_signal'], label='Signal', linewidth=1.5, color='red')
    ax3.bar(df.index, df['MACD_hist'], label='Histogram', alpha=0.6, color='gray')
    ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax3.set_ylabel('MACD', fontsize=12)
    ax3.legend(loc='best')
    ax3.grid(True, alpha=0.3)
    
    # 4. Volume
    ax4 = fig.add_subplot(gs[3])
    ax4.bar(df.index, df['volume'], alpha=0.6, color='steelblue')
    ax4.set_ylabel('Volume', fontsize=12)
    ax4.set_xlabel('Date', fontsize=12)
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    return fig

# Create visualizations for each stock
for ticker, df in indicators_data.items():
    print(f"Creating visualization for {ticker}...")
    fig = plot_stock_with_indicators(df, ticker, save_path=FIGURES_DIR / f'{ticker}_technical_analysis.png')
    plt.show()
    plt.close(fig)

print("\n✅ All visualizations created!")


In [ ]:
# Compare multiple stocks
if len(indicators_data) > 1:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Normalize prices to compare performance
    normalized_data = {}
    for ticker, df in indicators_data.items():
        normalized_data[ticker] = (df['close'] / df['close'].iloc[0]) * 100
    
    # 1. Normalized price comparison
    ax1 = axes[0, 0]
    for ticker, prices in normalized_data.items():
        ax1.plot(prices.index, prices.values, label=ticker, linewidth=2)
    ax1.set_ylabel('Normalized Price (Base=100)', fontsize=12)
    ax1.set_title('Normalized Price Comparison', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. RSI comparison
    ax2 = axes[0, 1]
    for ticker, df in indicators_data.items():
        ax2.plot(df.index, df['RSI'], label=ticker, linewidth=1.5, alpha=0.7)
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
    ax2.set_ylabel('RSI', fontsize=12)
    ax2.set_title('RSI Comparison', fontsize=14, fontweight='bold')
    ax2.set_ylim(0, 100)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Volatility comparison (if available)
    ax3 = axes[1, 0]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'volatility_30d' in metrics['data'].columns:
            ax3.plot(metrics['data'].index, metrics['data']['volatility_30d'] * 100, 
                    label=ticker, linewidth=1.5)
    ax3.set_ylabel('Volatility (%)', fontsize=12)
    ax3.set_title('30-Day Rolling Volatility', fontsize=14, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Returns distribution
    ax4 = axes[1, 1]
    for ticker, metrics in pynance_metrics.items():
        if 'data' in metrics and 'daily_return' in metrics['data'].columns:
            returns = metrics['data']['daily_return'].dropna() * 100
            ax4.hist(returns, bins=50, alpha=0.5, label=ticker, density=True)
    ax4.set_xlabel('Daily Return (%)', fontsize=12)
    ax4.set_ylabel('Density', fontsize=12)
    ax4.set_title('Daily Returns Distribution', fontsize=14, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'stocks_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Comparison visualization created!")


## 7. Summary and Key Insights


In [ ]:
print("=" * 70)
print("QUANTITATIVE ANALYSIS SUMMARY")
print("=" * 70)

print(f"\n📊 Stocks Analyzed: {len(indicators_data)}")
for ticker in indicators_data.keys():
    df = indicators_data[ticker]
    print(f"\n{ticker}:")
    print(f"  • Data points: {len(df):,}")
    print(f"  • Date range: {df.index.min().date()} to {df.index.max().date()}")
    print(f"  • Current price: ${df['close'].iloc[-1]:.2f}")
    print(f"  • Price change: {((df['close'].iloc[-1] / df['close'].iloc[0]) - 1) * 100:.2f}%")
    
    # RSI status
    current_rsi = df['RSI'].iloc[-1]
    if current_rsi > 70:
        rsi_status = "Overbought"
    elif current_rsi < 30:
        rsi_status = "Oversold"
    else:
        rsi_status = "Neutral"
    print(f"  • Current RSI: {current_rsi:.2f} ({rsi_status})")
    
    # MACD signal
    if df['MACD'].iloc[-1] > df['MACD_signal'].iloc[-1]:
        macd_signal = "Bullish"
    else:
        macd_signal = "Bearish"
    print(f"  • MACD signal: {macd_signal}")
    
    # Moving average trend
    if df['close'].iloc[-1] > df['SMA_50'].iloc[-1] > df['SMA_200'].iloc[-1]:
        trend = "Uptrend"
    elif df['close'].iloc[-1] < df['SMA_50'].iloc[-1] < df['SMA_200'].iloc[-1]:
        trend = "Downtrend"
    else:
        trend = "Sideways"
    print(f"  • Trend: {trend}")

if pynance_metrics:
    print(f"\n💰 Financial Metrics:")
    for ticker, metrics in pynance_metrics.items():
        if 'total_return' in metrics:
            print(f"\n{ticker}:")
            print(f"  • Total Return: {metrics['total_return']*100:.2f}%")
            if metrics.get('sharpe_ratio'):
                print(f"  • Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
            if metrics.get('max_drawdown'):
                print(f"  • Max Drawdown: {metrics['max_drawdown']*100:.2f}%")

print("\n" + "=" * 70)
print("✅ Quantitative Analysis Complete!")
print("=" * 70)
